In [1]:
description = '''
Using FINESSE and the galaxy page: https://galaxy.ligo.caltech.edu/optics/

Calculate the junk light or contrast defect from the antisymmetric port of the interferometer,
also calculate the necessary adaptive optical power to compensate.

TVo 2019-07-20

'''

In [2]:
import pykat
from pykat.ifo import aligo
import pykat.ifo.aligo.plot
import matplotlib.pyplot as plt
import numpy as np

#### My functions
import trace_mode

C:\Users\tvo88\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


                                              ..-
    PyKat 1.1.277         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



In [3]:
katfile = "aLIGO_IFO_AWC_tuning_DCoff_maxtem2.kat"

In [4]:
def calc_CD(overlap):
    CD = (1.0 - np.real(overlap) )/ 2.0
    return CD

In [5]:
class ifo_cavs(object):
    """ an IFO object that calculates the q for the various cavities in at three points
    ITMX, ITMY, and the Beamsplitter HR surface
    
    This will allow us to calculate the beam overlaps for the various modes.
    """
    def _init_(self, kat):
        None
        
    def calc_qs(self,kat):
        #### Turn off all the cav commands except XARM to extract the eigenmode at the BS
        [q_XARM_BS_x,q_XARM_BS_y] = trace_mode.from_cav_to_BS(kat,'cavXARM')

        #### Turn off all the cav commands except YARM to extract the eigenmode at the BS
        [q_YARM_BS_x,q_YARM_BS_y] = trace_mode.from_cav_to_BS(kat,'cavYARM')

        #### Turn off all cav commands except XARM and extrat mode at HR surface of ITMX
        [q_XARM_ITM_x,q_XARM_ITM_y]  = trace_mode.from_cav_to_node(kat,'cavXARM','nITMX2')

        #### Turn off all cav commands except YARM and extrat mode at HR surface of ITMY
        [q_YARM_ITM_x, q_YARM_ITM_y]  = trace_mode.from_cav_to_node(kat,'cavYARM','nITMY2')

        #### Turn off all cav commands except IMC and extrat mode at HR surface of ITMX
        [q_IMC_ITMX_x, q_IMC_ITMX_y]  = trace_mode.from_cav_to_node(kat,'cavIMC','nITMX2')

        #### Turn off all cav commands except IMC and extrat mode at HR surface of ITMY
        [q_IMC_ITMY_x, q_IMC_ITMY_y]  = trace_mode.from_cav_to_node(kat,'cavIMC','nITMY2')

         #### Turn off all cav commands except IMC and extrat mode at HR surface of ITMX
        [q_IMC_BS_x, q_IMC_BS_y]  = trace_mode.from_cav_to_node(kat,'cavIMC','nITMX2')

        #### Turn off all cav commands except PRX and extrat mode at HR surface of ITMX
        [q_PRX_ITMX_x,q_PRX_ITMX_y]  = trace_mode.from_cav_to_node(kat,'cavPRX','nITMX2')

        #### Turn off all cav commands except PRX and extrat mode at HR surface of ITMY
        [q_PRX_ITMY_x, q_PRX_ITMY_y]  = trace_mode.from_cav_to_node(kat,'cavPRX','nITMY2')
        
        #### Turn off all cav commands except PRX and extrat mode at HR surface of ITMX
        [q_PRY_ITMX_x,q_PRY_ITMX_y]  = trace_mode.from_cav_to_node(kat,'cavPRY','nITMX2')

        #### Turn off all cav commands except PRX and extrat mode at HR surface of ITMY
        [q_PRY_ITMY_x, q_PRY_ITMY_y]  = trace_mode.from_cav_to_node(kat,'cavPRY','nITMY2')
        
        q_dict = {"q_XARM_BS_x":q_XARM_BS_x,
                  "q_XARM_BS_y":q_XARM_BS_y,
                  "q_YARM_BS_x":q_YARM_BS_x,
                  "q_YARM_BS_y":q_YARM_BS_y,

                  "q_XARM_ITM_x":q_XARM_ITM_x,
                  "q_XARM_ITM_y":q_XARM_ITM_y,
                  "q_YARM_ITM_x":q_YARM_ITM_x,
                  "q_YARM_ITM_y":q_YARM_ITM_y,
                  
                  "q_IMC_ITMX_x":q_IMC_ITMX_x,
                  "q_IMC_ITMX_y":q_IMC_ITMX_y,
                  "q_IMC_ITMY_x":q_IMC_ITMY_x,
                  "q_IMC_ITMY_y":q_IMC_ITMY_y,
                
                  "q_PRX_ITMX_x":q_PRX_ITMX_x,
                  "q_PRX_ITMX_y":q_PRX_ITMX_y,
                  "q_PRX_ITMY_x":q_PRX_ITMY_x,
                  "q_PRX_ITMY_y":q_PRX_ITMY_y,
                
                  "q_PRY_ITMX_x":q_PRY_ITMX_x,
                  "q_PRY_ITMX_y":q_PRY_ITMX_y,
                  "q_PRY_ITMY_x":q_PRY_ITMY_x,
                  "q_PRY_ITMY_y":q_PRY_ITMY_y,
                
                  "q_IMC_BS_x":q_IMC_BS_x,
                  "q_IMC_BS_y":q_IMC_BS_y,
                 }
        
        return q_dict

In [6]:
#### Get the amplitude overlap between two gaussian beams
def amp_ovl(q1,q2):
    a = 2 * 1j * np.sqrt(np.imag(q1) * np.imag(q2)) / (q1-np.conjugate(q2))
    return a

In [7]:
### Calculate the required thermal power to change curvature
def calc_P(calib, R_new, R_old):
    
    P = 1/calib * ( 1/(R_new) - (1/R_old))
    
    return P

In [8]:
basekat = pykat.finesse.kat() 
basekat.load(katfile)
basekat.parse("noxaxis")
basekat.parse("maxtem "+str(2))
basekat.parse("""
ad SB9Mhz_plus 0 0 9.099471M nITMX2
ad SB9Mhz_minus 0 0 -9.099471M nITMX2

ad SB45Mhz_plus 0 0 45.497355M nITMX2
ad SB45Mhz_minus 0 0 -45.497355M nITMX2

bp ITMX_q_x x q nITMX2
bp ITMX_q_y y q nITMX2

bp ITMY_q_x x q nITMY2
bp ITMY_q_y y q nITMY2

pd REFL_port nPRBS

pd AS_port nSRBS

yaxis re:im
""")
kat=basekat.deepcopy() #copy kat code
kat.verbose = False
out = kat.run()

In [9]:
#### Set up the before parameters
kat.ITMXHR.Rcx.value = -1939.9
kat.ITMXHR.Rcy.value = -1939.9

kat.ITMYHR.Rcx.value = -1939.2
kat.ITMYHR.Rcy.value = -1939.2

kat.ETMXHR.Rcx.value = 2241.54 
kat.ETMXHR.Rcy.value = 2241.54 

kat.ETMYHR.Rcx.value = 2238.9
kat.ETMYHR.Rcy.value = 2238.9

Before = ifo_cavs().calc_qs(kat)

In [10]:
#### Set up the after parameters
kat1=basekat.deepcopy()
kat1.verbose=False

kat1.ITMXHR.Rcx.value = -1940.3
kat1.ITMXHR.Rcy.value = -1940.3

kat1.ITMYHR.Rcx.value = -1939.2
kat1.ITMYHR.Rcy.value = -1939.2

kat1.ETMXHR.Rcx.value = 2244.2 
kat1.ETMXHR.Rcy.value = 2244.2 

kat1.ETMYHR.Rcx.value = 2246.9
kat1.ETMYHR.Rcy.value = 2246.9

After = ifo_cavs().calc_qs(kat1)

In [11]:
before_overlap_ITM = amp_ovl(Before['q_XARM_ITM_x'],Before['q_YARM_ITM_x'])
before_overlap_BS = amp_ovl(Before['q_XARM_BS_x'],Before['q_YARM_BS_x'])

after_overlap_ITM = amp_ovl(After['q_XARM_ITM_x'],After['q_YARM_ITM_x'])
after_overlap_BS = amp_ovl(After['q_XARM_BS_x'],After['q_YARM_BS_x'])

In [12]:
After['q_XARM_ITM_x'],After['q_YARM_ITM_x']

((-1837.71148639769+434.198214896648j), (-1835.64863656683+435.986145537602j))

In [34]:
### Contrast defect at the ITMs with resonating arms
CD_old_ITM =calc_CD(before_overlap_ITM)
CD_new_ITM =calc_CD(after_overlap_ITM)

print(CD_old_ITM, CD_new_ITM)


RH_calib = .79e-6 #diopters per watt T1400685
P_RH_Arms_resonant = calc_P(RH_calib, -1940,-1939.2)
print("Required Thermal lens Power " + str(P_RH_Arms_resonant))

5.934299324494852e-06 3.865230617516868e-06
Required Thermal lens Power 0.2691773810030645


In [14]:
### Contrast defect at the BS (includes Schnupp Asymmetry and Thermal lens) with resonating arms
CD_old_BS = calc_CD(before_overlap_BS)
CD_new_BS = calc_CD(after_overlap_BS)
print(CD_old_BS, CD_new_BS)

5.683696534086735e-06 4.5933131715414355e-06


In [35]:
test_imc = calc_CD(amp_ovl(After['q_IMC_ITMX_x'],After['q_IMC_ITMY_y']))

In [39]:
### Propogate only the IMC Mode to the ITMs 
IMC_overlap_ITM = pykat.BeamParam.overlap(After['q_IMC_ITMX_x'],After['q_IMC_ITMY_y'])
IMC_CD = calc_CD(IMC_overlap_ITM)

#Modal CD for single bounce mode
print("IMC Contrast Defect in PPM " + str(IMC_CD*1e6))

R_IMC_ITMX = 1/np.real(1/After['q_IMC_ITMX_x'])
R_IMC_ITMY = 1/np.real(1/After['q_IMC_ITMY_x'])
P_IMC_mode = calc_P(RH_calib,R_IMC_ITMY,R_IMC_ITMX)
#print(R_IMC_ITMX,R_IMC_ITMY,P_IMC_mode)

IMC Contrast Defect in PPM 160.91119351957505


In [41]:
### Propogate only the PRX to the ITMX, and PRY to ITMY, this should estimate the overlap of the DRMI setup
PRC_overlap_ITM = pykat.BeamParam.overlap(After['q_PRX_ITMX_x'],After['q_PRY_ITMY_y'])

### Modal CD for DRMI
DRMI_CD = calc_CD(PRC_overlap_ITM)
print("DRMI Contrast Defect in PPM " + str(DRMI_CD*1e6))

DRMI Contrast Defect in PPM 186.45815364726204


In [19]:
#### Set up the after co2 parameters
kat2=basekat.deepcopy()
kat2.verbose=False

kat2.ITMXHR.Rcx.value = -1940.3
kat2.ITMXHR.Rcy.value = -1940.3

kat2.ITMYHR.Rcx.value = -1939.2
kat2.ITMYHR.Rcy.value = -1939.2

kat2.ETMXHR.Rcx.value = 2244.2 
kat2.ETMXHR.Rcy.value = 2244.2 

kat2.ETMYHR.Rcx.value = 2246.9
kat2.ETMYHR.Rcy.value = 2246.9

kat2.ITMXTL.f = 43300.0 # Guess

After_co2 = ifo_cavs().calc_qs(kat2)

In [20]:
### Propogate only the IMC Mode to the ITMs 
IMC_overlap_ITM = pykat.BeamParam.overlap(After_co2['q_IMC_ITMX_x'],After_co2['q_IMC_ITMY_y'])
IMC_CD = calc_CD(IMC_overlap_ITM)

#Modal CD for single bounce mode
print(IMC_CD*1e6)

R_IMC_ITMX = 1/np.real(1/After_co2['q_IMC_ITMX_x'])
R_IMC_ITMY = 1/np.real(1/After_co2['q_IMC_ITMY_x'])
P_IMC_mode = calc_P(RH_calib,R_IMC_ITMY,R_IMC_ITMX)
print(R_IMC_ITMX,R_IMC_ITMY,P_IMC_mode)

38.74079983823275
-1943.5142660992167 -1922.0134399380925 -7.285911698017736


In [21]:
### Propogate only the PRX to the ITMX, and PRY to ITMY, this should estimate the overlap of the DRMI setup
PRC_overlap_ITM_co2 = pykat.BeamParam.overlap(After_co2['q_PRX_ITMX_x'],After_co2['q_PRY_ITMY_y'])

### Modal CD for DRMI
DRMI_CD_co2 = calc_CD(PRC_overlap_ITM_co2)
print("Contrast Defect (PPM) is " + str(DRMI_CD_co2*1e6))

co2_calib = 62.3e-6 #diopters per watt

P_DRMI = calc_P(co2_calib, kat2.ITMXTL.f.value,kat1.ITMXTL.f.value)
print("Required Co2 power "+ str(P_DRMI))

Contrast Defect (PPM) is 0.7472762861282511
Required Co2 power -0.09455568257893743


In [23]:
#### Set up the after co2 parameters
kat3=basekat.deepcopy()
kat3.verbose=False

kat3.ITMXHR.Rcx.value = -1940.3
kat3.ITMXHR.Rcy.value = kat3.ITMXHR.Rcx.value

kat3.ITMYHR.Rcx.value = -1939.2
kat3.ITMYHR.Rcy.value = kat3.ITMYHR.Rcx.value

kat3.ETMXHR.Rcx.value = 2244.2 
kat3.ETMXHR.Rcy.value = kat3.ETMXHR.Rcx.value

kat3.ETMYHR.Rcx.value = 2246.9
kat3.ETMYHR.Rcy.value = kat3.ETMYHR.Rcx.value

for cav in kat3.getAll(pykat.commands.cavity):
    cav.enabled = False

#kat3.cavXARM.enabled = True
#kat3.cavYARM.enabled = True
kat3.cavIMC.enabled = True


kat3.parse('''
ad xTEM00 0 0 0 nITMX2
ad xTEM10 1 0 0 nITMX2
ad xTEM01 0 1 0 nITMX2
ad xTEM11 1 1 0 nITMX2
ad xTEM02 0 2 0 nITMX2
ad xTEM20 2 0 0 nITMX2

ad yTEM00 0 0 0 nITMY2
ad yTEM10 1 0 0 nITMY2
ad yTEM01 0 1 0 nITMY2
ad yTEM11 1 1 0 nITMY2
ad yTEM02 0 2 0 nITMY2
ad yTEM20 2 0 0 nITMY2
''')

X_Y_Arm = kat3.run()

In [31]:
x00,x02,x20 = np.abs(X_Y_Arm['xTEM00']),np.abs(X_Y_Arm['xTEM20']),np.abs(X_Y_Arm['xTEM02'])

In [32]:
CD = 1/ (4*(x00/x20) + 1)
print("Contrast defect at the antisymmetric port " + str(CD))

Contrast defect at the antisymmetric port 0.001226285042092446
